In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

In [ ]:
# Define the CNN model
def build_cnn_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
# Load and preprocess the data
def load_and_preprocess_data(train_dir, validation_dir, target_size=(150, 150), batch_size=32):
    train_datagen = ImageDataGenerator(rescale=1./255,
                                       rotation_range=40,
                                       width_shift_range=0.2,
                                       height_shift_range=0.2,
                                       shear_range=0.2,
                                       zoom_range=0.2,
                                       horizontal_flip=True,
                                       fill_mode='nearest')

    validation_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical')

    validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical')
    
    return train_generator, validation_generator

In [ ]:
# Main function to run the image recognition model
def run_image_recognition(train_dir, validation_dir):
    input_shape = (150, 150, 3) # Assuming color images with these dimensions
    num_classes = 10 # Update based on the number of classes in your dataset
    
    model = build_cnn_model(input_shape, num_classes)
    
    train_generator, validation_generator = load_and_preprocess_data(train_dir, validation_dir)
    
    history = model.fit(
        train_generator,
        steps_per_epoch=100, # Adjust based on the size of your dataset
        epochs=20,
        validation_data=validation_generator,
        validation_steps=50, # Adjust based on the size of your validation set
        verbose=2)
    
    
  # Plot training and validation accuracy and loss
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(acc))

    plt.plot(epochs, acc, 'r', label='Training accuracy')
    plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.legend()

    plt.figure()

    plt.plot(epochs, loss, 'r', label='Training Loss')
    plt.plot(epochs, val_loss, 'b', label='Validation Loss')
    plt.title('Training and validation loss')
    plt.legend()

    plt.show()

In [ ]:
if __name__ == "__main__":
    train_dir = 'path_to_train_directory'  # Update with the actual path to your training dataset
    validation_dir = 'path_to_validation_directory'  # Update with the actual path to your validation dataset
    run_image_recognition(train_dir, validation_dir)